In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import pandas as pd
from collections import deque

# === 配置参数 ===
class Config:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # 数据维度
    num_stocks = 30           # NSGA选出的股票数量
    seq_len = 250 * 5         # 回测窗口长度 (5年)
    feat_dim = 6              # 基础特征数 (Open, High, Low, Close, Vol, VWAP)
    context_dim = 16          # 股票池宏观特征维度 (PE分布, 行业向量等)

    # 网络参数
    d_model = 128
    n_heads = 4
    vib_beta = 0.01           # VIB正则化系数

    # 动作空间
    max_formula_len = 20      # 公式最大长度

# === 数据容器 ===
class StockData:
    def __init__(self):
        # 1. 行情张量: [Stocks, Time, Features]
        # Features 顺序: open, high, low, close, volume, vwap
        self.market_tensor = torch.randn(Config.num_stocks, Config.seq_len, Config.feat_dim).to(Config.device)

        # 2. 情境向量: [Context_Dim]
        # 包含：平均市盈率、平均波动率、行业集中度、大盘相关性等
        self.pool_context = torch.randn(Config.context_dim).to(Config.device)

        # 3. 股票代码索引
        self.codes = [f"s_{i:02d}" for i in range(Config.num_stocks)]

# 实例化数据
data_loader = StockData()